# ORMGP hydrological raster preparation
This notebook documents the creation of a set of derivative rasters needed for watershed characterization

All data can be found here: __*TO ADD*__

## Load land use data
The source of the land use data is: *Ministry of Natural Resources and Forestry, 2019. Southern Ontario Land Resource Information System (SOLRIS) Version 3.0: Data Specifications. Science and Research Branch, April 2019.*

The SOLRIS data has already been transformed to a 25,000x25,000, 10m grid that is here up-scaled to a 5000x5000, 50m grid. The following code collects the 5x5 10m sub-grids that are aggregated to every 50m cell:

In [1]:
import numpy as np
import imageio

s = np.fromfile("input/solrisv3_10_infilled.bil", np.int16).reshape(5000, 5, -1, 5).swapaxes(1,2).reshape(-1, 5*5)

Next, cross-referencing is accomplished to i) gather all unique SOLRIS IDs and ii) create an indexed grid referencing these unique IDs:

In [2]:
u, ix = np.unique(s, return_inverse=True)
ix = ix.reshape(s.shape)

The following line up-scales the SOLRIS IDs by assigning the dominant SOLRIS ID to every 50m cell, and saves the data:

In [3]:
s50 = u[np.argmax(np.apply_along_axis(np.bincount, 1, ix.reshape(-1, 5*5 ), None, np.max(ix) + 1), axis=1)].reshape(5000,5000)
s50.tofile("output/solrisv3_10_infilled_50.bil")
imageio.imwrite('output/solrisv3_10_infilled_50.png', s50)


Lossy conversion from int16 to uint8. Range [11, 250]. Convert image to uint8 prior to saving to suppress this warning.


### Compute aggregates
Next, up-scaling of land use properties (imperviousness, canopy cover, etc.) is computed by averaging the (25-cell sub-grid) 10m values over the 50m cell.

#### load lookup table:

In [4]:
import pandas as pd

df = pd.read_csv("input/lookup_200731.csv").fillna(0.0)
df['PerOW'] = np.select([df['Value'] == 170], [1.0], 0.0)
df['PerWL'] = np.select([(df['Value'] >= 131) & (df['Value'] <= 160)], [1.0], 0.0)

#### fraction of impervious cover:

In [5]:
# dimp = {i:0.0 for i in u}
# dimp[201] = .85
# dimp[202] = .1
# dimp[203] = .90
dimp = pd.Series(df.PerImp.values,index=df.Value).to_dict()

perimp = np.mean(np.vectorize(dimp.get)(s), 1).reshape(5000,5000)
perimp.tofile("output/solrisv3_10_infilled_50_perimp.bil")
imageio.imwrite('output/solrisv3_10_infilled_50_perimp.png', perimp)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


#### fraction of canopy cover:

In [6]:
dcov = pd.Series(df.PerCov.values,index=df.Value).to_dict()

percov = np.mean(np.vectorize(dcov.get)(s), 1).reshape(5000,5000)
percov.tofile("output/solrisv3_10_infilled_50_percov.bil")
imageio.imwrite('output/solrisv3_10_infilled_50_percov.png', percov)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


#### fraction water body cover:

In [7]:
dow = pd.Series(df.PerOW.values,index=df.Value).to_dict()

perow = np.mean(np.vectorize(dow.get)(s), 1).reshape(5000,5000)
perow.tofile("output/solrisv3_10_infilled_50_perow.bil")
imageio.imwrite('output/solrisv3_10_infilled_50_perow.png', perow)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


#### fraction wetland cover:

In [8]:
dwl = pd.Series(df.PerWL.values,index=df.Value).to_dict()

perwl = np.mean(np.vectorize(dwl.get)(s), 1).reshape(5000,5000)
perwl.tofile("output/solrisv3_10_infilled_50_perwl.bil")
imageio.imwrite('output/solrisv3_10_infilled_50_perwl.png', perwl)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


## Load surficial geology data
The source of the surficial geology data is: *Ontario Geological Survey 2010. Surficial geology of southern Ontario; Ontario Geological Survey, Miscellaneous Release— Data 128 – Revised.*

The raster loaded below has been build using QGIS, no further processing required here. The raster represents a [1,8] index referring to the "permeability" attribute (OGS, 2010).

In [10]:
g = np.fromfile("input/OGSsurfGeo_50.bil", np.int16).reshape(5000,5000)
imageio.imwrite('output/OGSsurfGeo_50.png', g)

Lossy conversion from int16 to uint8. Range [0, 8]. Convert image to uint8 prior to saving to suppress this warning.


Next, the permeability attributes are converted to an effective hydraulic conductivity, log-scaled:

In [11]:
dsg = {1: -9., 2: -8., 3: -7., 4: -6., 5: -5., 6: -8., 7: -5., 8: -6.}
gk = np.vectorize(dsg.get)(g)
imageio.imwrite('output/OGSsurfGeo_50-k.png', gk)

Lossy conversion from float64 to uint8. Range [-9.0, -5.0]. Convert image to uint8 prior to saving to suppress this warning.


# Sub-watershed aggregation
The ORMGP jurisdiction has been sub-divided into 2,813 ~10km² sub-watersheds and is provided as an index raster. Using a zonal analysis, raster values aggregated above are further aggregated to every sub-watershed.

#### SWS index raster and *"indicize"*:

In [12]:
w = np.fromfile("input/owrc20-50a_SWS10.bil", np.int32).reshape(5000,5000)
_, iw = np.unique(w, return_inverse=True)
imageio.imwrite('output/owrc20-50a_SWS10.png', w)

Lossy conversion from int32 to uint8. Range [-9999, 24685723]. Convert image to uint8 prior to saving to suppress this warning.


#### collect zonal summations and build dataframe:

In [13]:
n = np.unique(w, return_counts=True)
sgk = np.bincount(iw, weights=gk.flatten())
spi = np.bincount(iw, weights=perimp.flatten())
spc = np.bincount(iw, weights=percov.flatten())
sow = np.bincount(iw, weights=perow.flatten())
swl = np.bincount(iw, weights=perwl.flatten())

df = pd.DataFrame(data=dict(swsid=n[0],n=n[1],perm=sgk/n[1],perimp=spi/n[1],percov=spc/n[1],perow=sow/n[1],perwl=swl/n[1]))
df = df[df.swsid != -9999]

#### convert "effective" permeabilities back to OGS (2010) qualitative terms:

In [14]:
dperm = {-9.: 'low', -8.: 'low-medium', -7.: 'medium', -6.: 'medium-high', -5.: 'high'}
df.perm = round(df.perm,0)
df.replace({"perm": dperm}, inplace=True)

Next, this index raster is used to complete a *.csv* file that is to be joined/related to the sub-watershed polygon shapefile version of the raster loaded; this is accomplished in QGIS.

In [15]:
df.to_csv("output/owrc20-50a_SWS10-summary.csv",index=False)
print(df)

         swsid     n        perm    perimp    percov     perow     perwl
1      1088120  4833      medium  0.000000  0.889934  0.066360  0.058274
2      1223024  5062  low-medium  0.000000  0.869131  0.094943  0.047902
3      1318080  4615      medium  0.000000  0.875504  0.089309  0.046917
4      1492963  4383      medium  0.000000  0.931478  0.027899  0.054164
5      1628076  4105  low-medium  0.000000  0.918112  0.041657  0.053642
...        ...   ...         ...       ...       ...       ...       ...
2809  24445895  6393  low-medium  0.776088  0.044393  0.000000  0.000369
2810  24515876  4072  low-medium  0.596233  0.146953  0.000000  0.001798
2811  24535813  5594      medium  0.326551  0.476376  0.001580  0.021652
2812  24585744  4186  low-medium  0.067743  0.434102  0.000860  0.006756
2813  24685723  4003      medium  0.104090  0.557816  0.004177  0.019925

[2813 rows x 7 columns]
